In [3]:
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#"gemini-2.0-flash-thinking-exp-01-21"
#config = {'thinking_config': {'include_thoughts': True}}

#"gemini-2.0-flash-exp"

llm = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05", temperature=0)
df = pd.read_excel("query_result_2025-01-25T07_49_01.047868Z.xlsx")
df = df[df.abstract.notna()]

/usr/local/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
with open("prompt.txt", "r") as f:
    prompt = f.read()
prompt = PromptTemplate(input_variables=['PROPOSAL_INFO'],template=prompt)

chain = prompt | llm | JsonOutputParser()

proposal_info_columns = [ 'title'
  , 'abstract'
  , 'detailed_description'
  , 'outline'
  , 'objective']

In [6]:
result = []

In [7]:
#4 RPM
import time

processed_proposals = set(r.get('proposal_id') for r in result)
print(processed_proposals)

for proposal_id in df.proposal_id.unique():
    # Skip if already processed
    if proposal_id in processed_proposals:
        print(f"Skipping already processed proposal: {proposal_id}")
        continue
        
    print(f"Processing proposal: {proposal_id}")
    proposal_info = df[df.proposal_id == proposal_id][proposal_info_columns][:1].to_dict(orient='records')[0]
    proposal_info_str = [f'{x}: {proposal_info[x]}' for x in proposal_info_columns]
    chain = prompt | llm | JsonOutputParser()
    executive_method = chain.invoke({"PROPOSAL_INFO": proposal_info_str})
    
    # Add proposal_id to the result for tracking
    executive_method['proposal_id'] = proposal_id
    result.append(executive_method)
    time.sleep(20)

set()
Processing proposal: 2185846315039064832
Processing proposal: 2185844604283126528
Processing proposal: 2205136900467983104
Processing proposal: 2208409881428361984
Processing proposal: 2185842091785978624
Processing proposal: 2185845802830660352
Processing proposal: 2197952451796009728
Processing proposal: 2181661100230050560
Processing proposal: 2209040357629362944
Processing proposal: 2188525315913941760
Processing proposal: 2197262136667800320
Processing proposal: 2209120431347073792
Processing proposal: 2201815482904871680
Processing proposal: 2203075043099935488
Processing proposal: 2183455404749488896
Processing proposal: 2180814504890204928
Processing proposal: 2202583607111844608


In [9]:
from collections import Counter

Counter([ x['vote'] for x in result])


Counter({'+0': 11, '-0': 4, '+1': 1, '-1': 1})

# flash thinking

In [160]:
import os
from google import genai

client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"], http_options={'api_version':'v1alpha'})

config = {'thinking_config': {'include_thoughts': True}, 'temperature': 0}


In [162]:
thinking_result = []

for proposal_id in df.proposal_id.unique():
    print(proposal_id)
    proposal_info = df[df.proposal_id == proposal_id][proposal_info_columns][:1].to_dict(orient='records')[0]
    proposal_info_str = [f'{x}: {proposal_info[x]}' for x in proposal_info_columns]

    response = client.models.generate_content(
        model='gemini-2.0-flash-thinking-exp',
        contents=prompt.invoke(
            {"PROPOSAL_INFO": proposal_info_str}).text,
        config=config
)

    thinking_result.append(response)

2185846315039064832
2185844604283126528
2205136900467983104
2208409881428361984
2185842091785978624
2185845802830660352
2197952451796009728
2181661100230050560
2209040357629362944
2188525315913941760
2197262136667800320
2209120431347073792
2201815482904871680
2203075043099935488
2183455404749488896
2180814504890204928
2202583607111844608


In [169]:
print(thinking_result[0].text)

```json
{
  "comment": "提案主題關於 LLM 評估很有趣，但與 Python 的關聯性較弱。建議明確說明如何使用 Python 進行 LLM 實驗、數據分析或模型互動。例如，可以展示 Python 程式碼範例，介紹相關的 Python 函式庫，並深入探討 Python 在 LLM 評估中的技術細節。加強 Python 技術深度將使提案更符合 PyCon TW 主題。",
  "vote": "-0"
}
```


In [171]:
import json

parsed_result = [json.loads(x.text.strip('```json').strip('```').strip()) for x in thinking_result]

In [176]:
from collections import Counter
Counter([ x['vote'] for x in parsed_result])



Counter({'+0': 9, '-0': 4, '+1': 3, '-1': 1})

In [ ]:
Counter({'+0': 10, '-0': 4, '+1': 2, '-1': 1})
Counter({'+0': 9, '-0': 4, '+1': 3, '-1': 1}) #thinking

# Get result

In [10]:
pivot_df = df.pivot_table(
    index='proposal_id',
    columns=df.groupby('proposal_id').cumcount(),
    values='comment',
    aggfunc='first'
)
pivot_df.columns = [f'comment_{i+1}' for i in range(pivot_df.shape[1])]
pivot_df.reset_index(inplace=True)


In [11]:
# Create DataFrame from LLM results
regular_result_df = pd.DataFrame(result)

# Rename columns to distinguish results
regular_result_df = regular_result_df.add_prefix('LLM_')

proposal_df = df[['proposal_id'] + proposal_info_columns].drop_duplicates(subset=['proposal_id']).sort_values(by='proposal_id', ascending=False)

# pivot_df.merge(regular_result_df, left_on='proposal_id', right_on='LLM_proposal_id', how='left').merge(
#     proposal_df, on='proposal_id', how='left')

final_df = proposal_df.merge(regular_result_df, left_on='proposal_id', right_on='LLM_proposal_id', how='left').merge(
    pivot_df, on='proposal_id', how='left').sort_values(by='proposal_id', ascending=False)


In [12]:
final_df['human_eval'] = ''
final_df.to_excel('final_df_0208_pre.xlsx', index=False)

In [16]:
final_df[:3]

,proposal_id,title,abstract,detailed_description,outline,objective,LLM_comment,LLM_vote,LLM_proposal_id,comment_1,comment_2,comment_3,comment_4,comment_5,comment_6,comment_7,human_eval
0,2209120431347073792,Cracking the Code: Decoding Anti-Bot Systems!,Join me for a masterclass where I share the my...,Join me for a masterclass where I share the my...,"The presentation spans 35 minutes, followed by...","Web scraping is gaining momentum, particularly...","提案主題 ""Cracking the Code: Decoding Anti-Bot Sys...",+0,2209120431347073792,Thanks for this comprehensive proposal. I also...,I did look into the video talk. And together w...,NaN,NaN,NaN,NaN,NaN,
1,2209040357629362944,自動化電話錄音評測系統：提升超過 30 倍生產力,公司每天有上百通電話錄音需要進行稽核，以往僅能透過抽查的方式進行人工稽核，於是利用 STT(...,STT 工具：\r\nhttps://github.com/openai/whisper\r...,"5min 既有流程簡介\r\n5min 簡介 STT, LLM\r\n10min 程式碼分享...",幫助對於在客服或是電銷團隊能夠加速稽核流程，透過 Python 開源套件能在短時間建構出適合...,提案主題明確，展示了如何利用Python結合STT和LLM技術來自動化電話錄音評測，並顯著提...,+0,2209040357629362944,The outline is quite sparse and lacks substant...,這份提案介紹了一個自動化電話錄音評測系統，利用STT（Speech To Text）和LLM...,NaN,NaN,NaN,NaN,NaN,
2,2208409881428361984,Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢及報告生成平台,隨著企業越來越重視環境、社會和治理（ESG）的可持續發展目標，對於準確且全面的 ESG 資訊...,### Domain: ESG (環境、社會和治理)\r\nESG 指的是評估企業在環境保護...,* 引言：ESG的重要性、了解LLM與RAG [5 mins]\r\n * ESG對現...,本次演講目標是向以下受眾展示 LLM 和 RAG 技術在 ESG 資訊查詢和報告生成中的創新...,提案主題「Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢...,+0,2208409881428361984,看起來是在介紹LLM應用，但對於python主題看不出實際有何特殊處,The talk is undoubtedly fascinating and insigh...,NaN,NaN,NaN,NaN,NaN,


# append some info

In [36]:
vote_df = pd.read_excel('pycon_2025_raw_vote.xlsx', dtype={'vote': str})
vote_df['vote_int'] = vote_df['vote'].astype(int)

/usr/local/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [25]:
# Get multiple statistical measures for votes by proposal_id
vote_stats = vote_df.groupby('proposal_id', as_index=False)['vote'].agg([
    ('mode', lambda x: x.mode().iloc[0]),  # Most common vote
    ('mean', 'mean'),                      # Average vote
    ('std', 'std'),                        # Standard deviation
    ('count', 'count'),                    # Number of votes
])

In [26]:
final_df.merge(vote_stats, on='proposal_id', how='left')

#vote_stats

,proposal_id,title,abstract,detailed_description,outline,objective,LLM_comment,LLM_vote,LLM_proposal_id,comment_1,...,comment_3,comment_4,comment_5,comment_6,comment_7,human_eval,mode,mean,std,count
0,2209120431347073792,Cracking the Code: Decoding Anti-Bot Systems!,Join me for a masterclass where I share the my...,Join me for a masterclass where I share the my...,"The presentation spans 35 minutes, followed by...","Web scraping is gaining momentum, particularly...","提案主題 ""Cracking the Code: Decoding Anti-Bot Sys...",+0,2209120431347073792,Thanks for this comprehensive proposal. I also...,...,NaN,NaN,NaN,NaN,NaN,,0,0.214286,0.699293,14
1,2209040357629362944,自動化電話錄音評測系統：提升超過 30 倍生產力,公司每天有上百通電話錄音需要進行稽核，以往僅能透過抽查的方式進行人工稽核，於是利用 STT(...,STT 工具：\r\nhttps://github.com/openai/whisper\r...,"5min 既有流程簡介\r\n5min 簡介 STT, LLM\r\n10min 程式碼分享...",幫助對於在客服或是電銷團隊能夠加速稽核流程，透過 Python 開源套件能在短時間建構出適合...,提案主題明確，展示了如何利用Python結合STT和LLM技術來自動化電話錄音評測，並顯著提...,+0,2209040357629362944,The outline is quite sparse and lacks substant...,...,NaN,NaN,NaN,NaN,NaN,,0,-0.181818,0.404520,11
2,2208409881428361984,Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢及報告生成平台,隨著企業越來越重視環境、社會和治理（ESG）的可持續發展目標，對於準確且全面的 ESG 資訊...,### Domain: ESG (環境、社會和治理)\r\nESG 指的是評估企業在環境保護...,* 引言：ESG的重要性、了解LLM與RAG [5 mins]\r\n * ESG對現...,本次演講目標是向以下受眾展示 LLM 和 RAG 技術在 ESG 資訊查詢和報告生成中的創新...,提案主題「Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢...,+0,2208409881428361984,看起來是在介紹LLM應用，但對於python主題看不出實際有何特殊處,...,NaN,NaN,NaN,NaN,NaN,,0,-0.090909,0.539360,11
3,2205136900467983104,oh my gai - 你的 GAI 資訊助手,生成式人工智慧(Generative Artificial Intelligence，後文簡...,### 提案大綱詳細說明\r\n1. GAI 簡介: 介紹 GAI 目前的應用，及開發過程可...,### 30 分鐘 \r\n1. 簡介生成式 AI [5min] \r\n2. 蒐集資料與使...,對於 GAI 有興趣或者是在工作中有使用到 GAI 的人來說，本演講將分享一個運用 GAI ...,"提案主題 ""oh my gai - 你的 GAI 資訊助手"" 圍繞著利用生成式 AI (GA...",+0,2205136900467983104,感覺上和python的關係較低，更像是在介紹AI應用而已,...,NaN,NaN,NaN,NaN,NaN,,0,-0.090909,0.539360,11
4,2203075043099935488,Query your structured data with a LangChain AI...,"In this talk, we will explore the capabilities...",LangChain: https://python.langchain.com/docs/g...,1. Intro to LLMs and prompt [5 min]\r\n2. What...,The intended audience of this talk are Python ...,提案主題與Python相關，介紹了LangChain這個用於開發LLM應用程式的框架。提案內...,+0,2203075043099935488,LangChain 已經問世一年左右了，有好也有壞處。\r\n提案中未涉及LangChain...,...,NaN,NaN,NaN,NaN,NaN,,0,0.000000,0.471405,10
5,2202583607111844608,Driving Efficiency through Automation: Leverag...,Efficiently tackling some of the resource prov...,"During this session, I'll provide a comprehens...",1. Introduction: (2 minutes)\r\n . Briefly ...,By codifying our infrastructure configurations...,提案主題與Python相關，介紹如何使用Python CDK在AWS上進行基礎設施自動化。提...,+0,2202583607111844608,I like the topic of infrastructure as code and...,...,NaN,NaN,NaN,NaN,NaN,,1,0.400000,0.699206,10
6,2201815482904871680,Building Retrieval Augmented Generation (RAG) ...,Retrieval Augmented Generation (RAG) has emerg...,What is RAG: https://medium.com/building-the-o...,1. Intro to LLMs and prompt [10 min]\r\n2. Wha...,The intended audience of this talk are Python ...,提案主題與大型語言模型 (LLM) 應用相關，並介紹了使用 LangChain 框架建構 R...,+0,2201815482904871680,1. 技術創新度：\r\n雖然涉及使用 LangChain 構建RAG應用，這是目前在自然語...,...,NaN,NaN,NaN,NaN,NaN,,0,-0.100000,0.316228,10
7,2197952451796009728,"Introduction to Threads, Processes, Coroutines...",We typically execute our Python programs seque...,- [Threads](https://docs.python.org/3/library/...,- Introduction\r\n- Concurrency and Parallelis...,The python developers want to learn more about...,提案涵蓋了 Python 中並行和並發的重要概念，包括執行緒、行程和協程，並結合了講者的實際...,+1,2197952451796009728,round 2 update: 提案者沒有改善提案。\r\n\r\n\r\n==\r\n1....,...,NaN,NaN,NaN,NaN,NaN,,0,0.000000,0.500000,9
8,2197262136667800320,利用 LangChain 與 FastAPI 實現 LLM 數位孿生系統,聊天機器人是大語言模型產品（如 ChatGPT 等）的常見使用界面，被視為一種對話式的使用者...,講者將以自己開發的開源框架 (https://github.com/towningtek/L...,- 對話式的使用者介面 (Conversational User Interface (CU...,Python 開發者在面對大型語言模型（LLM）等生成式 AI 專案開發時，經常面臨提示工程...,提案主題圍繞著使用 LangChain 和 FastAPI 構建基於 LLM 的數位孿生系統...,+0,2197262136667800320,W

In [39]:
final_df

,proposal_id,title,abstract,detailed_description,outline,objective,LLM_comment,LLM_vote,LLM_proposal_id,comment_1,comment_2,comment_3,comment_4,comment_5,comment_6,comment_7,human_eval,human_mode,agreement
0,2209120431347073792,Cracking the Code: Decoding Anti-Bot Systems!,Join me for a masterclass where I share the my...,Join me for a masterclass where I share the my...,"The presentation spans 35 minutes, followed by...","Web scraping is gaining momentum, particularly...","提案主題 ""Cracking the Code: Decoding Anti-Bot Sys...",+0,2209120431347073792,Thanks for this comprehensive proposal. I also...,I did look into the video talk. And together w...,NaN,NaN,NaN,NaN,NaN,,NaN,False
1,2209040357629362944,自動化電話錄音評測系統：提升超過 30 倍生產力,公司每天有上百通電話錄音需要進行稽核，以往僅能透過抽查的方式進行人工稽核，於是利用 STT(...,STT 工具：\r\nhttps://github.com/openai/whisper\r...,"5min 既有流程簡介\r\n5min 簡介 STT, LLM\r\n10min 程式碼分享...",幫助對於在客服或是電銷團隊能夠加速稽核流程，透過 Python 開源套件能在短時間建構出適合...,提案主題明確，展示了如何利用Python結合STT和LLM技術來自動化電話錄音評測，大幅提升...,+0,2209040357629362944,The outline is quite sparse and lacks substant...,這份提案介紹了一個自動化電話錄音評測系統，利用STT（Speech To Text）和LLM...,NaN,NaN,NaN,NaN,NaN,,NaN,False
2,2208409881428361984,Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢及報告生成平台,隨著企業越來越重視環境、社會和治理（ESG）的可持續發展目標，對於準確且全面的 ESG 資訊...,### Domain: ESG (環境、社會和治理)\r\nESG 指的是評估企業在環境保護...,* 引言：ESG的重要性、了解LLM與RAG [5 mins]\r\n * ESG對現...,本次演講目標是向以下受眾展示 LLM 和 RAG 技術在 ESG 資訊查詢和報告生成中的創新...,提案主題「Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢...,+0,2208409881428361984,看起來是在介紹LLM應用，但對於python主題看不出實際有何特殊處,The talk is undoubtedly fascinating and insigh...,NaN,NaN,NaN,NaN,NaN,,NaN,False
3,2205136900467983104,oh my gai - 你的 GAI 資訊助手,生成式人工智慧(Generative Artificial Intelligence，後文簡...,### 提案大綱詳細說明\r\n1. GAI 簡介: 介紹 GAI 目前的應用，及開發過程可...,### 30 分鐘 \r\n1. 簡介生成式 AI [5min] \r\n2. 蒐集資料與使...,對於 GAI 有興趣或者是在工作中有使用到 GAI 的人來說，本演講將分享一個運用 GAI ...,"提案主題""oh my gai - 你的 GAI 資訊助手"" 具有實用性，針對當前 GAI 資...",+0,2205136900467983104,感覺上和python的關係較低，更像是在介紹AI應用而已,"While your platform ""oh my gai"" is an interest...",NaN,NaN,NaN,NaN,NaN,,NaN,False
4,2203075043099935488,Query your structured data with a LangChain AI...,"In this talk, we will explore the capabilities...",LangChain: https://python.langchain.com/docs/g...,1. Intro to LLMs and prompt [5 min]\r\n2. What...,The intended audience of this talk are Python ...,提案主題與Python相關，介紹了LangChain這個用於開發LLM應用程式的框架。提案大...,+0,2203075043099935488,LangChain 已經問世一年左右了，有好也有壞處。\r\n提案中未涉及LangChain...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,False
5,2202583607111844608,Driving Efficiency through Automation: Leverag...,Efficiently tackling some of the resource prov...,"During this session, I'll provide a comprehens...",1. Introduction: (2 minutes)\r\n . Briefly ...,By codifying our infrastructure configurations...,提案主題與 Python 相關，介紹了如何使用 Python CDK 進行 AWS 基礎設施...,+0,2202583607111844608,I like the topic of infrastructure as code and...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,False
6,2201815482904871680,Building Retrieval Augmented Generation (RAG) ...,Retrieval Augmented Generation (RAG) has emerg...,What is RAG: https://medium.com/building-the-o...,1. Intro to LLMs and prompt [10 min]\r\n2. Wha...,The intended audience of this talk are Python ...,提案主題與大型語言模型 (LLM) 相關，並介紹了使用 LangChain 框架構建 RAG...,-0,2201815482904871680,1. 技術創新度：\r\n雖然涉及使用 LangChain 構建RAG應用，這是目前在自然語...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,False
7,2197952451796009728,"Introduction to Threads, Processes, Coroutines...",We typically execute our Python programs seque...,- [Threads](https://docs.python.org/3/library/...,- Introduction\r\n- Concurrency and Parallelis...,The python developers want to learn more about...,"提案主題明確，涵蓋了 Python 中併發與平行處理的重要概念：Threads, Proce...",+0,2197952451796009728,round 2 update: 提案者沒有改善提案。\r\n\r\n\r\n==\r\n1....,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,False
8,2197262136667800320,利用 LangChain 與 FastAPI 實現 LLM 數位孿生系統,聊天機器人是大語言模型產品（如 ChatGPT 等）的常見使用界面，被視為一種對話式的使用者...,講者將以自己開發的開源框架 (https://github.com/towningtek/L...,- 對話式的使用者介面 (Conversational User Interface (CU...,Python 開發者在面對大型語言模型（LLM）等生成式 AI 專案

In [35]:
from statistics import mode

# Get unique proposal IDs that exist in both datasets
common_proposals = set(vote_df['proposal_id'].unique()) & set(proposal_id for r in result)

# Create lists of equal length
proposal_ids = sorted(list(common_proposals))
human_modes = []
ai_predictions = []

for pid in proposal_ids:
    # Get human mode for this proposal - directly calculate mode from raw votes
    human_mode = mode(vote_df[vote_df['proposal_id'] == pid]['vote'].tolist())
    human_modes.append(human_mode)
    
    # Get AI prediction for this proposal
    ai_pred = next(r['vote'] for r in result if r['proposal_id'] == pid)
    ai_predictions.append(ai_pred)

# Create comparison DataFrame with aligned data
comparison_df = pd.DataFrame({
    'proposal_id': proposal_ids,
    'human_mode': human_modes,
    'ai_prediction': ai_predictions
})

# Calculate agreement rate
agreement_count = (comparison_df['human_mode'] == comparison_df['ai_prediction']).sum()
agreement_rate = agreement_count / len(comparison_df)

print(f"AI 與人類的主流意見一致率: {agreement_rate:.2f}")

# Add these metrics to final_df
final_df['human_mode'] = final_df['proposal_id'].map(
    comparison_df.set_index('proposal_id')['human_mode'])
final_df['agreement'] = final_df['human_mode'] == final_df['LLM_vote']

AI 與人類的主流意見一致率: 0.00


In [36]:
final_df

,proposal_id,title,abstract,detailed_description,outline,objective,LLM_comment,LLM_vote,LLM_proposal_id,comment_1,comment_2,comment_3,comment_4,comment_5,comment_6,comment_7,human_eval,human_mode,agreement
0,2209120431347073792,Cracking the Code: Decoding Anti-Bot Systems!,Join me for a masterclass where I share the my...,Join me for a masterclass where I share the my...,"The presentation spans 35 minutes, followed by...","Web scraping is gaining momentum, particularly...","提案主題 ""Cracking the Code: Decoding Anti-Bot Sys...",+0,2209120431347073792,Thanks for this comprehensive proposal. I also...,I did look into the video talk. And together w...,NaN,NaN,NaN,NaN,NaN,,NaN,False
1,2209040357629362944,自動化電話錄音評測系統：提升超過 30 倍生產力,公司每天有上百通電話錄音需要進行稽核，以往僅能透過抽查的方式進行人工稽核，於是利用 STT(...,STT 工具：\r\nhttps://github.com/openai/whisper\r...,"5min 既有流程簡介\r\n5min 簡介 STT, LLM\r\n10min 程式碼分享...",幫助對於在客服或是電銷團隊能夠加速稽核流程，透過 Python 開源套件能在短時間建構出適合...,提案主題明確，展示了如何利用Python結合STT和LLM技術來自動化電話錄音評測，大幅提升...,+0,2209040357629362944,The outline is quite sparse and lacks substant...,這份提案介紹了一個自動化電話錄音評測系統，利用STT（Speech To Text）和LLM...,NaN,NaN,NaN,NaN,NaN,,NaN,False
2,2208409881428361984,Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢及報告生成平台,隨著企業越來越重視環境、社會和治理（ESG）的可持續發展目標，對於準確且全面的 ESG 資訊...,### Domain: ESG (環境、社會和治理)\r\nESG 指的是評估企業在環境保護...,* 引言：ESG的重要性、了解LLM與RAG [5 mins]\r\n * ESG對現...,本次演講目標是向以下受眾展示 LLM 和 RAG 技術在 ESG 資訊查詢和報告生成中的創新...,提案主題「Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢...,+0,2208409881428361984,看起來是在介紹LLM應用，但對於python主題看不出實際有何特殊處,The talk is undoubtedly fascinating and insigh...,NaN,NaN,NaN,NaN,NaN,,NaN,False
3,2205136900467983104,oh my gai - 你的 GAI 資訊助手,生成式人工智慧(Generative Artificial Intelligence，後文簡...,### 提案大綱詳細說明\r\n1. GAI 簡介: 介紹 GAI 目前的應用，及開發過程可...,### 30 分鐘 \r\n1. 簡介生成式 AI [5min] \r\n2. 蒐集資料與使...,對於 GAI 有興趣或者是在工作中有使用到 GAI 的人來說，本演講將分享一個運用 GAI ...,"提案主題""oh my gai - 你的 GAI 資訊助手"" 具有實用性，針對當前 GAI 資...",+0,2205136900467983104,感覺上和python的關係較低，更像是在介紹AI應用而已,"While your platform ""oh my gai"" is an interest...",NaN,NaN,NaN,NaN,NaN,,NaN,False
4,2203075043099935488,Query your structured data with a LangChain AI...,"In this talk, we will explore the capabilities...",LangChain: https://python.langchain.com/docs/g...,1. Intro to LLMs and prompt [5 min]\r\n2. What...,The intended audience of this talk are Python ...,提案主題與Python相關，介紹了LangChain這個用於開發LLM應用程式的框架。提案大...,+0,2203075043099935488,LangChain 已經問世一年左右了，有好也有壞處。\r\n提案中未涉及LangChain...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,False
5,2202583607111844608,Driving Efficiency through Automation: Leverag...,Efficiently tackling some of the resource prov...,"During this session, I'll provide a comprehens...",1. Introduction: (2 minutes)\r\n . Briefly ...,By codifying our infrastructure configurations...,提案主題與 Python 相關，介紹了如何使用 Python CDK 進行 AWS 基礎設施...,+0,2202583607111844608,I like the topic of infrastructure as code and...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,False
6,2201815482904871680,Building Retrieval Augmented Generation (RAG) ...,Retrieval Augmented Generation (RAG) has emerg...,What is RAG: https://medium.com/building-the-o...,1. Intro to LLMs and prompt [10 min]\r\n2. Wha...,The intended audience of this talk are Python ...,提案主題與大型語言模型 (LLM) 相關，並介紹了使用 LangChain 框架構建 RAG...,-0,2201815482904871680,1. 技術創新度：\r\n雖然涉及使用 LangChain 構建RAG應用，這是目前在自然語...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,False
7,2197952451796009728,"Introduction to Threads, Processes, Coroutines...",We typically execute our Python programs seque...,- [Threads](https://docs.python.org/3/library/...,- Introduction\r\n- Concurrency and Parallelis...,The python developers want to learn more about...,"提案主題明確，涵蓋了 Python 中併發與平行處理的重要概念：Threads, Proce...",+0,2197952451796009728,round 2 update: 提案者沒有改善提案。\r\n\r\n\r\n==\r\n1....,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,False
8,2197262136667800320,利用 LangChain 與 FastAPI 實現 LLM 數位孿生系統,聊天機器人是大語言模型產品（如 ChatGPT 等）的常見使用界面，被視為一種對話式的使用者...,講者將以自己開發的開源框架 (https://github.com/towningtek/L...,- 對話式的使用者介面 (Conversational User Interface (CU...,Python 開發者在面對大型語言模型（LLM）等生成式 AI 專案

In [26]:
df.vote.unique()

array([-1])